Let's start off by importing some established and custom functions and classes

In [ ]:
# math and stats
import math
import numpy as np
from scipy.stats import binned_statistic, spearmanr
from scipy import interpolate

# image processing
from skimage import io, filters, measure, morphology
from skimage.external import tifffile
from scipy.ndimage import center_of_mass
from scipy.ndimage.morphology import distance_transform_edt

#plotting tools
import matplotlib.pyplot as plt
from matplotlib import patches as mp
from matplotlib import cm



def rad_from_points(x1, y1, x2, y2, x3, y3):
    '''
    Given three points (x1, y1), (x2, y2), (x3, y3), return the radius r and center (xc, yc)
    of the circle passing through these three points
    '''
    ma = (y2 - y1)/(x2 - x1)
    mb = (y3 - y2)/(x3 - x2)
    
    xc = (ma*mb*(y1 - y3) + mb*(x1 + x2) - ma*(x2 + x3))/(2*(mb - ma))
    yc = -1/ma*(xc - (x1 + x2)/2) + (y1 + y2)/2
    
    if ma == mb:
        r = np.inf
    else:
        r = np.hypot(xc - x1, yc - y1)
    
    return(r, xc, yc)
    

def radius_of_curvature(x_path, y_path, scale):
    '''
    Given a closed path described by points x_path, y_path and a scale factor,
    return an array of radii of signed curvature along the closed path.
    
    The scale factor determines how many indices to the left and to the right of 
    a point of interest to consider in determining a circle whose arc approximates the
    path curvature near the point of interest.
    '''
    
    r = []
    xcs = []
    ycs = []
    num_points = len(x_path)
    for i in range(num_points):
        # points
        x1 = x_path[i - scale]
        y1 = y_path[i - scale]
        x2 = x_path[i]
        y2 = y_path[i]
        x3 = x_path[(i + scale)%num_points]
        y3 = y_path[(i + scale)%num_points]
        
        # fit circle
        rad, xc, yc = rad_from_points(x1, y1, x2, y2, x3, y3)
        
        # get vector normal to path for sign of curvature
        nv1 = np.cross(np.array([x2 - x1, y2 - y1, 0]), np.array([0 ,0, 1]))
        nv2 = np.cross(np.array([x3 - x2, y3 - y2, 0]), np.array([0 ,0, 1]))
        
        nv = np.average([nv1, nv2], axis = 0)
        
        # get sign of dot product 
        align = np.sign(np.dot(nv[0:2], np.array([x2 - xc, y2 - yc])))
        
        
        theta = np.linspace(-math.pi, math.pi, 100)
        x_plot = rad * np.cos(theta) + xc
        y_plot = rad * np.sin(theta) + yc
        
        if rad == 0:
            r.append(np.nan)
        else:
            r.append(align * 1./rad)
            
        xcs.append(xc)
        ycs.append(yc)
        
    return(r, xcs, ycs)


def signed_distmap(binary_img):
    '''
    Useful function for generating a distance function that maps pixel locations
    to distance from the edge(s) of the input binary image. Points on the interior
    are negative and points outside of the region are positive. By tracing the cell
    bounary at time t+1 for the signed distance map of the boundary at time t and
    collecting interpolated values along that path, we get a measure of minimum cell
    boundary displacement.
    '''
    A = distance_transform_edt(binary_img.astype(bool))
    B = distance_transform_edt(~binary_img.astype(bool))
    return(B - A)

def generate_velmap(bin_stack):
    '''
    This function uses the signed distance map described above to estimate the
    instantaneous velocity of the cell boundary at all points using forward and
    backward differences
    '''

    delta_distmap = []

    delta_distmap.append(np.zeros_like(bin_stack[0]))
    for t in range(1, len(bin_stack)-1):
        result = 0.5 * (signed_distmap(bin_stack[t-1]) - signed_distmap(bin_stack[t + 1]))
        delta_distmap.append(filters.gaussian(result, 1, preserve_range = True))
    delta_distmap.append(np.zeros_like(bin_stack[0]))

    return(np.array(delta_distmap))

Next, we'll read in our images of interest (Cells with TEMs)

In [ ]:
parent = '/Users/jason/Projects/wavecomplex_selforganization/data/raw_data/'

filename_1 = '190406_HUVEC_Nap-eGFO_Y27_1002-1.tif'
im_1 = io.imread(parent + filename_1)
t, c, w, h = im_1.shape
print(t, c, w, h)

filename_2 = '190326_HUVEC_Nap1-eGFP_Y27_1002-1.tif'
im_2 = io.imread(parent + filename_2)
t, c, w, h = im_2.shape
print(t, c, w, h)

microns_per_pixel = 0.183
time_per_frame_im1 = 5.0
time_per_frame_im2 = 2.0

And let's visualize the images themselves and some thresholds we'll use to segment the cells from the background using a simple intensity-based approach

In [ ]:
plt.figure(figsize = (10,3))
plt.subplot(121)
plt.hist(im_1[0,0].ravel(), bins = np.linspace(0, 1000, 101))

plt.subplot(122)
plt.hist(im_2[0,0].ravel(), bins = np.linspace(0, 1000, 101))
plt.show()

plt.figure(figsize = (10,10))

plt.subplot(121)
plt.imshow(im_1[0,0], vmin = 100, vmax = 800)
binary_im = filters.gaussian(im_1[0,0], preserve_range = True) > 140
binary_im = morphology.remove_small_objects(binary_im, 50)
plt.contour(binary_im, levels = [False], colors = 'w')
plt.axis('off')

plt.subplot(122)
binary_im = filters.gaussian(im_2[0,0], preserve_range = True) > 110
binary_im = morphology.remove_small_objects(binary_im, 50)
plt.imshow(im_2[0,0], vmin = 100, vmax = 400)
plt.contour(binary_im, levels = [False], colors = 'w')
plt.axis('off')

plt.show()


plt.figure(figsize = (10,10))

plt.subplot(121)
plt.imshow(im_1[10,0], vmin = 100, vmax = 800)
binary_im = filters.gaussian(im_1[10,0], preserve_range = True) > 140
binary_im = morphology.remove_small_objects(binary_im, 50)
plt.contour(binary_im, levels = [False], colors = 'w')
plt.axis('off')

plt.subplot(122)
plt.imshow(im_2[10,0], vmin = 100, vmax = 400)
binary_im = filters.gaussian(im_2[10,0], preserve_range = True) > 110
binary_im = morphology.remove_small_objects(binary_im, 50)
plt.contour(binary_im, levels = [False], colors = 'w')
plt.axis('off')

plt.show()


This segmentation strategy looks like it does an ok job at identifying TEMs. Let's continue by taking a closer look

## TEM 1

In [ ]:
# zoom in on a closing TEM

count = 1

plt.figure(figsize = (10,8))

for t in range(0, 100, 5): # look at every 5th frame
    ax = plt.subplot(4, 5, count)
    

    crop = im_2[t, 0, 170:250, 340:420] # zoom in on one region
    crop_smth = filters.gaussian(crop, preserve_range = True)
    plt.imshow(crop, vmin = 100, vmax = 400)
    ctrs = measure.find_contours(crop_smth, 130)

    for i in ctrs:
        plt.plot(i[:,1], i[:,0], color = 'w')
        
    ax.text(10, 10, 'F' + str(t), color = 'r', size = 12, fontweight = 'bold', ha = 'center', va = 'center')
    plt.xticks([])
    plt.yticks([])
    count += 1
    
plt.tight_layout()
plt.show()

# zoom in on a closing TEM

count = 1

plt.figure(figsize = (10,8))

for t in range(0, 100, 5): # look at every 5th frame
    ax = plt.subplot(4, 5, count)
    

    crop = im_2[t, 0, 170:250, 340:420] # zoom in on one region
    crop_smth = filters.gaussian(crop, preserve_range = True)
    crop2 = im_2[t, 1, 170:250, 340:420]
    plt.imshow(crop2, vmin = 100, vmax = 160)
    ctrs = measure.find_contours(crop_smth, 130)

    for i in ctrs:
        plt.plot(i[:,1], i[:,0], color = 'w')
        
    ax.text(10, 10, 'F' + str(t), color = 'r', size = 12, fontweight = 'bold', ha = 'center', va = 'center')
    plt.xticks([])
    plt.yticks([])
    count += 1
    
plt.tight_layout()
plt.show()

Let's work toward isolating TEMs from spurious background. We'll do the selection manually by choosing which continuous regions to further analyze.

In [ ]:
# smooth out and binarize the cropped area
smoothed_stack = [filters.gaussian(crop, preserve_range = True)[170:250, 340:420] for crop in im_2[:,0]]
smoothed_stack = np.array(smoothed_stack)
crop_bin = (smoothed_stack < 130)

# apply a label in 3d to isolate temporospatially-connected regions
crop_lbl = measure.label(crop_bin)

# apply crop to the wrc channel as well
wave_stack = im_2[:, 1, 170:250, 340:420]


for t in range(0, 100, 5):
    
    plt.figure(figsize = (5,5))
    ax = plt.subplot()

    reg_props = measure.regionprops(crop_lbl[t])

    plt.imshow(im_2[t, 0, 170:250, 340:420], vmin = 100, vmax = 400)

    ctrs = measure.find_contours(smoothed_stack[t], 130)

    for i in ctrs:
        plt.plot(i[:,1], i[:,0], color = 'w')

    for c in range(len(reg_props)):
        L = reg_props[c].label
        y, x = reg_props[c].centroid

        ax.text(x, y, L, color = 'w', fontweight = 'bold')

    plt.axis('off')

    plt.show()

In [ ]:
closing_coords = [center_of_mass(crop_lbl[t] == 1) for t in range(99)]
closing_coords_b = [center_of_mass(crop_lbl[t] == 5) for t in range(99)]

for t in range(90):
    
    plt.figure(figsize = (9, 3))
    
    ax = plt.subplot(131)
    plt.imshow(im_2[t, 0], vmin = 100, vmax = 400)
    
    rect = mp.Rectangle((340, 170), 80, 80, edgecolor = 'white', fill = None)
    ax.add_patch(rect)
    
    rect = mp.Rectangle((10, 10), 40/microns_per_pixel, 16, color = 'white')
    ax.text(120 , 56, '40 um', fontsize = 16, color = 'white', fontweight = 'bold', ha = 'center', va = 'center')
    ax.add_patch(rect)
    plt.title('Membrane Channel', fontweight = 'bold')
    plt.xticks([])
    plt.yticks([])
    
    ax = plt.subplot(132)

    reg_props = measure.regionprops(crop_lbl[t])

    plt.imshow(im_2[t, 0, 170:250, 340:420], vmin = 100, vmax = 400)

    ctrs = measure.find_contours(smoothed_stack[t], 130)

    for i in ctrs:
        x_m = np.average(i[:,1])
        y_m = np.average(i[:,0])


        y_L, x_L = closing_coords[t]
        y_Lb, x_Lb = closing_coords_b[t]

        if np.hypot(y_m - y_L, x_m - x_L) < 10:
            plt.plot(i[:,1], i[:,0], color = 'w', lw = 2, alpha = 0.8)
            
        #if np.hypot(y_m - y_Lb, x_m - x_Lb) < 10:
            #plt.plot(i[:,1], i[:,0], color = 'w', linestyle = '--', lw = 2, alpha = 0.8)

    rect = mp.Rectangle((1, 1), 5/microns_per_pixel, 2, color = 'white')
    ax.text(15 , 7, '5 um', fontsize = 16, color = 'white', fontweight = 'bold', ha = 'center', va = 'center')
    ax.add_patch(rect)
        
    plt.xticks([])
    plt.yticks([])
    plt.title('Membrane Channel (zoom)', fontweight = 'bold')
    
    ax2 = plt.subplot(133)
    
    plt.imshow(im_2[t, 1, 170:250, 340:420], vmin = 100, vmax = 160)

    ctrs = measure.find_contours(smoothed_stack[t], 130)

    for i in ctrs:
        x_m = np.average(i[:,1])
        y_m = np.average(i[:,0])


        y_L, x_L = closing_coords[t]
        y_Lb, x_Lb = closing_coords_b[t]

        if np.hypot(y_m - y_L, x_m - x_L) < 10:
            plt.plot(i[:,1], i[:,0], color = 'w', lw = 2, alpha = 0.8)
            
        #if np.hypot(y_m - y_Lb, x_m - x_Lb) < 10:
            #plt.plot(i[:,1], i[:,0], color = 'w', linestyle = '--', lw = 2, alpha = 0.8)
            
    
    ax2.text(74, 74, str(time_per_frame_im2 * t) + ' s', fontsize = 16, color = 'white', fontweight = 'bold', ha = 'right', va = 'center')
    plt.title('Nap1 Channel (zoom)', fontweight = 'bold')
    
    plt.xticks([])
    plt.yticks([])

    plt.tight_layout()
    plt.show()
    

Using this simple manual instance segmentation strategy, we've isolated the pixels associated with a single closing TEM. From here, we can start to analyze fluorescence and geometric properties of our isolated TEM. For example, we can look at the relative area of the TEM above as a function of time (in frames) and the average fluorescence signal in a donut-shaped selection around the TEM boundary. Here, we can see that Wave complex enriches to the TEM at the same time that it is closing. This could indicate that WRC helps cause TEM closure or that WRC localized to closing TEMs, possibly due to geometric features of the closing TEM. 

In [ ]:



TEM_OF_INTEREST = 1

# We can now easily extract a temporal profile of the area of our TEM of interest
area_tem = [np.sum(crop_lbl[t] == TEM_OF_INTEREST) for t in range(99)]

# approximate the background fluorescence in the wrc channel
wrc_bg = np.median(wave_stack[crop_lbl == TEM_OF_INTEREST])

# We can also extract information about the fluorescence around our TEM of interest
fluor_tem = []

tem_present = np.where([np.sum(crop_lbl[t] == TEM_OF_INTEREST) > 0 for t in range(len(crop_lbl))])[0]
for t in tem_present:
    
    hole = (crop_lbl[t] == TEM_OF_INTEREST)
    
    if np.sum(hole) > 0:
        contracted = morphology.binary_erosion(hole, selem = morphology.disk(2))
        expanded = morphology.binary_dilation(hole, selem = morphology.disk(5))
        ring = expanded & ~contracted
        fluor_tem.append(np.average(wave_stack[t][ring]))
        
# take median intensity value inside the TEM as camera background value for WRC fluorescence
# a better background value might be taken from further inside the cell, but this would require more assumptions
fluor_bg = np.nanmedian([np.median(wave_stack[t][crop_lbl[t] == 1]) for t in range(99)])

plt.figure(figsize = (10, 4))
ax = plt.subplot()
plt.plot(time_per_frame_im2 * np.arange(len(area_tem)), np.array(area_tem) * (microns_per_pixel)**2, lw = 2, color = 'C1')
plt.ylabel('Area (square microns)', fontweight = 'bold', fontsize = 16, color = 'C1')
plt.xlabel('Time (Seconds)', fontweight = 'bold', fontsize = 16)
plt.ylim(0, 12)

ax.twinx()

plt.plot((time_per_frame_im2 * np.arange(len(area_tem)))[tem_present], np.array(fluor_tem) - fluor_bg, lw = 2, color = 'C2')
plt.ylabel('Avg WRC Fluorescence\n(Units above background)', fontweight = 'bold', fontsize = 16, color = 'C2')
plt.ylim(0, 25)

plt.show()

While these global features of the TEMs are interesting in their own right, we can also extract sub-TEM information to examine the dynamic relationship between local TEM geometry and WRC localization patterns

In [ ]:

results = []


#########################
# Parameters!
#########################
which_TEM = 1
angular_resolution = 50
#########################

wave_stack = im_2[:, 1, 170:250, 340:420]
memb_stack = im_2[:, 0, 170:250, 340:420]

# approximate the background fluorescence in the wrc channel
wrc_bg = np.median(wave_stack[crop_lbl == TEM_OF_INTEREST])
mem_bg = np.median(memb_stack[crop_lbl == TEM_OF_INTEREST])

# collect a binary map and time coordinates of when tem exists
tem_bin_map = np.array([crop_lbl[t] == which_TEM for t in range(len(crop_lbl))])
tem_exists_in_frame = np.where([np.sum(crop_lbl[t] == which_TEM) > 0 for t in range(len(crop_lbl))])[0]

# get coordinates for the center of the TEM over time
closing_coords = [center_of_mass(tem_bin_map[t]) for t in tem_exists_in_frame]

# we also have the option to stabilize the TEM in case translational movement confounds the results
centered_bin_map = [np.roll(tem_bin_map[t], 40 - np.round(closing_coords[t][0]).astype(int), axis = 0) for t in tem_exists_in_frame]
centered_bin_map = [np.roll(centered_bin_map[t], 40 - np.round(closing_coords[t][1]).astype(int), axis = 1) for t in tem_exists_in_frame]
centered_bin_map = np.array(centered_bin_map)

vmap = -generate_velmap(tem_bin_map.astype(int))
t, s1, s2 = crop_lbl.shape

angles = []
curvs = []
velos = []
fluor = []
x_positions = []
y_positions = []
memcon = []

plt.figure(figsize = (10,10), facecolor = 'w')
plt.subplot(aspect = 'equal')

cmap = cm.get_cmap('viridis')
col_vec = [cmap(i) for i in np.linspace(0,1,99)]

for t in tem_exists_in_frame:
    
    # find smooth contours using the find_contours function
    ctrs = measure.find_contours(smoothed_stack[t], 130)

    for i in ctrs:
        x_m = np.average(i[:,1])
        y_m = np.average(i[:,0])

        y_L, x_L = closing_coords[t]
        
        # find the smooth contour that corresponds to the TEM of interest (which is stored as a binary image)
        # note that the find_contours function has jagged features when applied to binary images. However,
        # these binary maps are *generally* easier to work with, hence the switching between the two
        if np.hypot(y_m - y_L, x_m - x_L) < 5:

            y, x = i.T
            
            # add extra points to our curve without changing its shape too much
            tck, u = interpolate.splprep([x, y], s = 0)
            param_new = np.linspace(0, 1, len(x) * 5) 
            x_new, y_new = interpolate.splev(param_new, tck)
        
            # find the radii of curvature along these newly created point-rich paths
            r, xcs, ycs = radius_of_curvature(x_new * microns_per_pixel, y_new * microns_per_pixel, 10)
            r = -np.array(r) # flip sign due to reversed orientation of segmented region compared to lamellipod
            
            # estimate velocity along the point-rich path from the minimum distances between edges at 
            # consecutive timepoints in microns/minute
            v = interpolate.interpn((np.arange(s1), np.arange(s2)), vmap[t] * microns_per_pixel/time_per_frame_im2 * 60, np.array([y_new, x_new]).T)
            
            # translate points to origin for angle calculations
            xcentered = x_new - np.average(x_new)
            ycentered = y_new - np.average(y_new)
            
            a = np.arctan2(-ycentered, xcentered)
            
            # bin collected info on radii of curvature, velocity, x and y positions as function of angle from center of TEM
            m, edge, binnum = binned_statistic(a[np.isfinite(r)], r[np.isfinite(r)], statistic = 'mean', bins = np.linspace(-math.pi, math.pi, angular_resolution + 1))
            curvs.append(m)
            
            m, edge, binnum = binned_statistic(a[np.isfinite(v)], v[np.isfinite(v)], statistic = 'mean', bins = np.linspace(-math.pi, math.pi, angular_resolution + 1))
            velos.append(m)
            
            m, edge, binnum = binned_statistic(a, x_new, statistic = 'mean', bins = np.linspace(-math.pi, math.pi, angular_resolution + 1))
            x_positions.append(m)
            
            m, edge, binnum = binned_statistic(a, y_new, statistic = 'mean', bins = np.linspace(-math.pi, math.pi, angular_resolution + 1))
            y_positions.append(m)
            
            if t < len(tem_exists_in_frame) - 1:
                plt.scatter(x_new, y_new, c = r, vmin = -4, vmax = 4, cmap = 'coolwarm', alpha = 0.5)
                
    # WRC data must be collected a little differently than the geometric data above
    # by creating a donut-shaped region around the boundary of the TEM
    contracted = morphology.binary_erosion(tem_bin_map[t], selem = morphology.disk(2))
    expanded = morphology.binary_dilation(tem_bin_map[t], selem = morphology.disk(5))
    ring = expanded & ~contracted
    
    # and collecting angles between points in the ring and the center of the TEM
    y_vals_ring, x_vals_ring = np.where(ring)
    x_L = np.average(x_vals_ring.astype(float))
    y_L = np.average(y_vals_ring.astype(float))
    x_vals_centered = x_vals_ring.astype(float) - x_L
    y_vals_centered = y_vals_ring.astype(float) - y_L

    angle = np.arctan2(-y_vals_centered, x_vals_centered)

    # we can summarize the angularly resolved WRC signal around the edge of the TEM
    m, edge, binnum = binned_statistic(angle, wave_stack[t][ring] - wrc_bg, statistic = 'mean', bins = np.linspace(-math.pi, math.pi, angular_resolution + 1))
    s, edge, binnum = binned_statistic(angle, wave_stack[t][ring] - wrc_bg, statistic = 'std', bins = np.linspace(-math.pi, math.pi, angular_resolution + 1))
    fluor.append(m)
    
    m, edge, binnum = binned_statistic(angle, memb_stack[t][ring] - mem_bg, statistic = 'mean', bins = np.linspace(-math.pi, math.pi, angular_resolution + 1))
    s, edge, binnum = binned_statistic(angle, memb_stack[t][ring] - mem_bg, statistic = 'std', bins = np.linspace(-math.pi, math.pi, angular_resolution + 1))
    memcon.append(m)
    
# convert collected data into numpy array for convenience
curvs = np.array(curvs)
velos = np.array(velos)
fluor = np.array(fluor)
memcon = np.array(memcon)
            
plt.colorbar()
            
plt.axis('off')

plt.show()



In [ ]:
from scipy import stats 

plt.subplot(111)
plt.title('Spatiotemporal Dynamics\nof Membrane Signal on TEM of Interest', fontweight = 'bold')
plt.imshow(memcon, vmin = 0, vmax = 100, origin = 'lower', aspect = 1, interpolation = 'nearest')
plt.colorbar(shrink = 0.3, label = 'Membrane Siganl Intensity\n(above background)')

xtick_pos = np.arange(0, angular_resolution + 1, angular_resolution/10) - 0.5
xtick_lab = np.linspace(0, 360, len(xtick_pos))
plt.xticks(xtick_pos, xtick_lab, rotation = 'vertical')
plt.show()

plt.figure(figsize = (10, 6), facecolor = 'w')

plt.subplot(131)
plt.title('Spatiotemporal Dynamics\nof WRC on TEM of Interest', fontweight = 'bold')
plt.imshow(fluor, vmin = 0, vmax = 40, origin = 'lower', aspect = 1, interpolation = 'nearest')
plt.colorbar(shrink = 0.3, label = 'WRC Intensity\n(above background)')

xtick_pos = np.arange(0, angular_resolution + 1, angular_resolution/10) - 0.5
xtick_lab = np.linspace(0, 360, len(xtick_pos))
plt.xticks(xtick_pos, xtick_lab, rotation = 'vertical')

plt.xlabel('Angle from Center\n(Degrees)', fontweight = 'bold')
plt.ylabel('Time (frames)', fontweight = 'bold')

plt.subplot(132)
plt.title('Spatiotemporal Dynamics\nof Curvature on TEM of Interest', fontweight = 'bold')
plt.imshow(curvs, vmin = -4, vmax = 4, cmap = 'coolwarm', origin = 'lower', aspect = 1, interpolation = 'nearest')
plt.colorbar(shrink = 0.3, label = '1/Radius of Curvature (1/um)')

xtick_pos = np.arange(0, angular_resolution + 1, angular_resolution/10) - 0.5
xtick_lab = np.linspace(0, 360, len(xtick_pos))
plt.xticks(xtick_pos, xtick_lab, rotation = 'vertical')
plt.xlabel('Angle from Center\n(Degrees)', fontweight = 'bold')
plt.ylabel('Time (frames)', fontweight = 'bold')

plt.subplot(133)
plt.title('Spatiotemporal Dynamics\nof Velocity on TEM of Interest', fontweight = 'bold')
plt.imshow(velos, vmin = -10, vmax = 10, cmap = 'coolwarm', origin = 'lower', aspect = 1, interpolation = 'nearest')
plt.colorbar(shrink = 0.3, label = 'Velocity (microns/minute)')

xtick_pos = np.arange(0, angular_resolution + 1, angular_resolution/10) - 0.5
xtick_lab = np.linspace(0, 360, len(xtick_pos))
plt.xticks(xtick_pos, xtick_lab, rotation = 'vertical')
plt.xlabel('Angle from Center\n(Degrees)', fontweight = 'bold')
plt.ylabel('Time (frames)', fontweight = 'bold')

plt.tight_layout()
plt.show()

m = memcon[:-1]
w = fluor[:-1]
c = curvs[:-1]
dc = np.diff(curvs, axis = 0)

b = np.linspace(0, 60, 31)

plt.hist((w[(c > 0) & (dc < 0)]), bins = b, density = True, color = 'blue', alpha = 0.5)
plt.hist((w[(c > 0) & (dc > 0)]), bins = b, density = True, color = 'red', alpha = 0.5)

a = np.log10(w[(c > 0) & (dc < 0)])
b = np.log10(w[(c > 0) & (dc > 0)])
print(stats.ttest_ind(a[np.isfinite(a)], b[np.isfinite(b)]))

plt.show()

fold_enrich = np.nanmean(w[(c > 0) & (dc < 0)]) / np.nanmean(w[(c > 0) & (dc > 0)])
print(fold_enrich)

T, AR = w.shape


fold_rot_func_1 = []
fold_rot_func_1_control = []

for i in range(-AR//2, AR//2 + 1):
    w_rotated = np.roll(w, i, axis = 1)
    m_rotated = np.roll(m, i, axis = 1)
    
    fold_enrich_null = np.nanmean(w_rotated[(c > 0) & (dc < 0)]) / np.nanmean(w_rotated[(c > 0) & (dc > 0)])
    fold_rot_func_1.append(fold_enrich_null)
    
    fold_enrich_null_control = np.nanmean(m_rotated[(c > 0) & (dc < 0)]) / np.nanmean(m_rotated[(c > 0) & (dc > 0)])
    fold_rot_func_1_control.append(fold_enrich_null_control)
    


results.append([fold_rot_func_1, fold_rot_func_1_control])
    
plt.axhline(1, ls = '--', color = 'k')
plt.plot(np.linspace(-180, 180, AR + 1), fold_rot_func_1, label = 'WRC')
plt.plot(np.linspace(-180, 180, AR + 1), fold_rot_func_1_control, label = 'Membrane')
plt.ylabel('Fold Enrichment of Signal in\nflattening vs. lagging regions', fontweight = 'bold', fontsize = 15)
plt.xlabel('Rotation', fontweight = 'bold', fontsize = 15)
plt.xlim(-180, 180)
plt.ylim(0.8, 1.2)
plt.legend()
plt.show()

plt.subplot(111)
plt.title('Spatiotemporal Dynamics\nof Membrane Signal on TEM of Interest', fontweight = 'bold')
plt.imshow(fluor/memcon, vmin = 0, vmax = 1, origin = 'lower', aspect = 1, interpolation = 'nearest')
plt.colorbar(shrink = 0.3, label = 'Membrane-Normalized WRC Signal Intensity\n(above background)')

xtick_pos = np.arange(0, angular_resolution + 1, angular_resolution/10) - 0.5
xtick_lab = np.linspace(0, 360, len(xtick_pos))
plt.xticks(xtick_pos, xtick_lab, rotation = 'vertical')
plt.show()

plt.figure(figsize = (6,6))
plt.title('Spatiotemporal Dynamics\nof Curvature on TEM of Interest', fontweight = 'bold')
plt.imshow(curvs, vmin = -4, vmax = 4, cmap = 'coolwarm', origin = 'lower', aspect = 1)
plt.colorbar(shrink = 0.5, label = 'Curvature')

wave_bin_map = filters.gaussian(fluor, preserve_range = True) > 15
plt.contour(wave_bin_map, levels = [False], colors = 'k', linestyles = '--')
plt.contourf(wave_bin_map, levels = [0.5, 1], hatches = ['..'], colors='none')

xtick_pos = np.arange(0, angular_resolution + 1, angular_resolution/10) - 0.5
xtick_lab = np.linspace(0, 360, len(xtick_pos))
plt.xticks(xtick_pos, xtick_lab, rotation = 'vertical')
plt.xlabel('Angle from Center\n(Degrees)', fontweight = 'bold')
plt.ylabel('Time (frames)', fontweight = 'bold')
plt.tight_layout()
plt.show()

In [ ]:

plt.scatter(fluor[1:-1], velos[1:-1], marker = '.', alpha = 0.1, color = 'C0')
plt.ylabel('Edge Velocity', fontweight = 'bold', fontsize = 15)
plt.xlabel('WRC Enrichment', fontweight = 'bold', fontsize = 15)
plt.axhline(0, ls = '--', color = 'k')
#plt.ylim(-15,15)
#plt.xlim(-10, 70)
plt.show()

plt.scatter(memcon[1:-1], velos[1:-1], marker = '.', alpha = 0.1, color = 'C0')
plt.ylabel('Edge Velocity', fontweight = 'bold', fontsize = 15)
plt.xlabel('Membrane Signal Enrichment', fontweight = 'bold', fontsize = 15)
#plt.ylim(-15,15)
#plt.xlim(-10, 350)
plt.show()

plt.scatter(fluor[1:-1]/memcon[1:-1], velos[1:-1], marker = '.', alpha = 0.1, color = 'C0')
plt.ylabel('Edge Velocity', fontweight = 'bold', fontsize = 15)
plt.xlabel('Normalized WRC Signal Enrichment', fontweight = 'bold', fontsize = 15)
plt.axhline(0, ls = '--', color = 'k')
#plt.ylim(-15,15)
#plt.xlim(-0.2, 1)
plt.show()

x = fluor[1:-1]/memcon[1:-1]
y = velos[1:-1]
print(spearmanr(x[np.isfinite(x) * np.isfinite(y)],y[np.isfinite(x) * np.isfinite(y)] ))

tem1_x = x
tem1_y = y


w = fluor[1:-1]/memcon[1:-1]
v = velos[1:-1]

T, AR = w.shape
fold_rot_func = []
for i in range(-AR//2, AR//2 + 1):
    w_rotated = np.roll(w, i, axis = 1)
    
    x = w_rotated
    y = v
    
    spearman_corr = spearmanr(x[np.isfinite(x) * np.isfinite(y)],y[np.isfinite(x) * np.isfinite(y)] )[0]
    fold_rot_func.append(spearman_corr)
    
    
plt.plot(np.linspace(-180, 180, AR + 1), fold_rot_func)
plt.ylabel('Spearman Correlation', fontweight = 'bold', fontsize = 15)
plt.xlabel('Rotation of Wave Complex Signal', fontweight = 'bold', fontsize = 15)
plt.xlim(-180, 180)
plt.ylim(-1, 1)
plt.show()

## TEM 2

In [ ]:
plt.figure(figsize = (10,10))
plt.imshow(im_1[t, 0], vmin = 100, vmax = 500)
plt.grid()
plt.show()

In [ ]:
for t in range(0, 90, 5):
    plt.figure(figsize = (10,5))
    ax = plt.subplot(121)
    ymin = 285
    xmin = 330
    plt.imshow(im_1[t, 0], vmin = 100, vmax = 500)
    rect = mp.Rectangle((xmin, ymin), 80, 80, edgecolor = 'white', fill = None)
    ax.add_patch(rect)
    #plt.grid()
    
    plt.subplot(122)
    plt.imshow(im_1[t, 0, ymin:ymin + 80, xmin:xmin + 80], vmin = 100, vmax = 500)
    
    plt.tight_layout()
    plt.show()

In [ ]:
# zoom in on a closing TEM

count = 1

thresh = 200

plt.figure(figsize = (10,8))

for t in range(0, 100, 5): # look at every 5th frame
    ax = plt.subplot(4, 5, count)
    
    im_1[t, 0, ]
    crop = im_1[t, 0, ymin:ymin+80, xmin:xmin+80] # zoom in on one region
    crop_smth = filters.gaussian(crop, preserve_range = True)
    plt.imshow(crop, vmin = 100, vmax = 400)
    ctrs = measure.find_contours(crop_smth, thresh)

    for i in ctrs:
        plt.plot(i[:,1], i[:,0], color = 'w')
        
    ax.text(10, 10, 'F' + str(t), color = 'r', size = 12, fontweight = 'bold', ha = 'center', va = 'center')
    plt.xticks([])
    plt.yticks([])
    count += 1
    
plt.tight_layout()
plt.show()

# zoom in on a closing TEM

count = 1

plt.figure(figsize = (10,8))

for t in range(0, 100, 5): # look at every 5th frame
    ax = plt.subplot(4, 5, count)
    

    crop = im_1[t, 0, ymin:ymin+80, xmin:xmin+80] # zoom in on one region
    crop_smth = filters.gaussian(crop, preserve_range = True)
    crop2 = im_1[t, 1, ymin:ymin+80, xmin:xmin+80]
    plt.imshow(crop2, vmin = 100, vmax = 160)
    ctrs = measure.find_contours(crop_smth, thresh)

    for i in ctrs:
        plt.plot(i[:,1], i[:,0], color = 'w')
        
    ax.text(10, 10, 'F' + str(t), color = 'r', size = 12, fontweight = 'bold', ha = 'center', va = 'center')
    plt.xticks([])
    plt.yticks([])
    count += 1
    
plt.tight_layout()
plt.show()

smoothed_stack = [filters.gaussian(crop, preserve_range = True)[ymin:ymin+80, xmin:xmin+80] for crop in im_1[:,0]]
smoothed_stack = np.array(smoothed_stack)
crop_bin = (smoothed_stack < thresh)

crop_lbl = measure.label(crop_bin)

wave_stack = im_1[:, 1, ymin:ymin+80, xmin:xmin+80]


In [ ]:

for t in range(0, 100, 5):
    
    plt.figure(figsize = (5,5))
    ax = plt.subplot()

    reg_props = measure.regionprops(crop_lbl[t])

    plt.imshow(im_1[t, 0, ymin:ymin+80, xmin:xmin+80], vmin = 100, vmax = 400)

    ctrs = measure.find_contours(smoothed_stack[t], thresh)

    for i in ctrs:
        plt.plot(i[:,1], i[:,0], color = 'w')

    for c in range(len(reg_props)):
        L = reg_props[c].label
        y, x = reg_props[c].centroid

        ax.text(x, y, L, color = 'w', fontweight = 'bold')

    plt.axis('off')

    plt.show()

In [ ]:
TEM_OF_INTEREST = 2
closing_coords = [center_of_mass(crop_lbl[t] == TEM_OF_INTEREST) for t in range(99)]


for t in range(90):
    
    plt.figure(figsize = (9, 3))
    
    ax = plt.subplot(131)
    plt.imshow(im_1[t, 0], vmin = 100, vmax = 600)
    plt.xticks([])
    plt.yticks([])
    rect = mp.Rectangle((xmin, ymin), 80, 80, edgecolor = 'white', fill = None)
    ax.add_patch(rect)
    
    rect = mp.Rectangle((10, 10), 40/microns_per_pixel, 16, color = 'white')
    ax.text(130 , 75, '40 um', fontsize = 16, color = 'white', fontweight = 'bold', ha = 'center', va = 'center')
    ax.add_patch(rect)
    
    ax = plt.subplot(132)

    reg_props = measure.regionprops(crop_lbl[t])

    plt.imshow(im_1[t, 0, ymin:ymin+80, xmin:xmin+80], vmin = 100, vmax = 600)

    ctrs = measure.find_contours(smoothed_stack[t], thresh)

    for i in ctrs:
        x_m = np.average(i[:,1])
        y_m = np.average(i[:,0])


        y_L, x_L = closing_coords[t]
        y_Lb, x_Lb = closing_coords_b[t]

        if np.hypot(y_m - y_L, x_m - x_L) < 5:
            plt.plot(i[:,1], i[:,0], color = 'w', lw = 2, alpha = 0.8)

    rect = mp.Rectangle((1, 1), 5/microns_per_pixel, 2, color = 'white')
    ax.text(15 , 7, '5 um', fontsize = 16, color = 'white', fontweight = 'bold', ha = 'center', va = 'center')
    ax.add_patch(rect)
        
    plt.axis('off')
    
    ax2 = plt.subplot(133)
    
    plt.imshow(im_1[t, 1,  ymin:ymin+80, xmin:xmin+80], vmin = 100, vmax = 160)

    ctrs = measure.find_contours(smoothed_stack[t], thresh)

    for i in ctrs:
        x_m = np.average(i[:,1])
        y_m = np.average(i[:,0])


        y_L, x_L = closing_coords[t]
        y_Lb, x_Lb = closing_coords_b[t]

        if np.hypot(y_m - y_L, x_m - x_L) < 10:
            plt.plot(i[:,1], i[:,0], color = 'w', lw = 2, alpha = 0.8)
            
        #if np.hypot(y_m - y_Lb, x_m - x_Lb) < 10:
            #plt.plot(i[:,1], i[:,0], color = 'w', linestyle = '--', lw = 2, alpha = 0.8)
            
    
    ax2.text(74, 74, str(np.round(t * 5, 1)) + ' s', fontsize = 16, color = 'white', fontweight = 'bold', ha = 'right', va = 'center')
    plt.axis('off')

    plt.tight_layout()
    
    plt.show()
    


In [ ]:
TEM_OF_INTEREST = 2

# We can now easily extract a temporal profile of the area of our TEM of interest
area_tem = [np.sum(crop_lbl[t] == TEM_OF_INTEREST) for t in range(99)]

# approximate the background fluorescence in the wrc channel
wrc_bg = np.median(wave_stack[crop_lbl == TEM_OF_INTEREST])

# We can also extract information about the fluorescence around our TEM of interest
fluor_tem = []

tem_present = np.where([np.sum(crop_lbl[t] == TEM_OF_INTEREST) > 0 for t in range(len(crop_lbl))])[0]
for t in tem_present:
    
    hole = (crop_lbl[t] == TEM_OF_INTEREST)
    
    if np.sum(hole) > 0:
        contracted = morphology.binary_erosion(hole, selem = morphology.disk(2))
        expanded = morphology.binary_dilation(hole, selem = morphology.disk(5))
        ring = expanded & ~contracted
        fluor_tem.append(np.average(wave_stack[t][ring]))
        
# take median intensity value inside the TEM as camera background value for WRC fluorescence
# a better background value might be taken from further inside the cell, but this would require more assumptions
fluor_bg = np.nanmedian([np.median(wave_stack[t][crop_lbl[t] == 1]) for t in range(99)])

plt.figure(figsize = (10, 4))
ax = plt.subplot()
timepoints_sec = time_per_frame_im1 * np.arange(len(area_tem))
plt.plot(timepoints_sec, np.array(area_tem) * microns_per_pixel**2, lw = 2, color = 'C1')
plt.ylabel('Area (Square Microns)', fontweight = 'bold', fontsize = 16, color = 'C1')
plt.xlabel('Time (Seconds)', fontweight = 'bold', fontsize = 16)
plt.ylim(0, 16)

ax.twinx()

plt.plot(timepoints_sec[tem_present], np.array(fluor_tem) - fluor_bg, lw = 2, color = 'C2')
plt.ylabel('Avg WRC Fluorescence\n(Units above background)', fontweight = 'bold', fontsize = 16, color = 'C2')
plt.ylim(0, 25)

plt.show()

In [ ]:

for TEM_OF_INTEREST in [2]:
    #########################
    # Parameters!
    #########################

    angular_resolution = 50
    #########################

    wave_stack = im_1[:, 1, ymin:ymin+80, xmin:xmin+80]
    memb_stack = im_1[:, 0, ymin:ymin+80, xmin:xmin+80]

    # approximate the background fluorescence in the wrc channel
    wrc_bg = np.median(wave_stack[crop_lbl == TEM_OF_INTEREST])
    mem_bg = np.median(memb_stack[crop_lbl == TEM_OF_INTEREST])

    # collect a binary map and time coordinates of when tem exists
    tem_bin_map = np.array([crop_lbl[t] == TEM_OF_INTEREST for t in range(len(crop_lbl))])
    tem_exists_in_frame = np.where([np.sum(crop_lbl[t] == TEM_OF_INTEREST) > 0 for t in range(len(crop_lbl))])[0]

    # get coordinates for the center of the TEM over time
    closing_coords = [center_of_mass(tem_bin_map[t]) for t in range(len(tem_bin_map))]

    # we also have the option to stabilize the TEM in case translational movement confounds the results
    #centered_bin_map = [np.roll(tem_bin_map[t], 40 - np.round(closing_coords[t][0]).astype(int), axis = 0) for t in tem_exists_in_frame]
    #centered_bin_map = [np.roll(centered_bin_map[t], 40 - np.round(closing_coords[t][1]).astype(int), axis = 1) for t in tem_exists_in_frame]
    #centered_bin_map = np.array(centered_bin_map)

    vmap = -generate_velmap(tem_bin_map.astype(int))
    t, s1, s2 = crop_lbl.shape

    angles = []
    curvs = []
    velos = []
    fluor = []
    x_positions = []
    y_positions = []
    memcon = []

    plt.figure(figsize = (10,10), facecolor = 'w')
    plt.subplot(aspect = 'equal')

    cmap = cm.get_cmap('viridis')
    col_vec = [cmap(i) for i in np.linspace(0,1,99)]

    for t in tem_exists_in_frame:

        # find smooth contours using the find_contours function
        ctrs = measure.find_contours(smoothed_stack[t], thresh)

        for i in ctrs:
            x_m = np.average(i[:,1])
            y_m = np.average(i[:,0])

            y_L, x_L = closing_coords[t]

            # find the smooth contour that corresponds to the TEM of interest (which is stored as a binary image)
            # note that the find_contours function has jagged features when applied to binary images. However,
            # these binary maps are *generally* easier to work with, hence the switching between the two
            if np.hypot(y_m - y_L, x_m - x_L) < 5:

                y, x = i.T

                # add extra points to our curve without changing its shape too much
                tck, u = interpolate.splprep([x, y], s = 0)
                param_new = np.linspace(0, 1, len(x) * 5) 
                x_new, y_new = interpolate.splev(param_new, tck)

                # find the radii of curvature along these newly created point-rich paths
                r, xcs, ycs = radius_of_curvature(x_new * microns_per_pixel, y_new * microns_per_pixel, 10)
                r = -np.array(r) # flip sign due to reversed orientation of segmented region compared to lamellipod

                # estimate velocity along the point-rich path from the minimum distances between edges at 
                # consecutive timepoints in microns/minute
                v = interpolate.interpn((np.arange(s1), np.arange(s2)), vmap[t] * microns_per_pixel/time_per_frame_im1 * 60, np.array([y_new, x_new]).T)

                # translate points to origin for angle calculations
                xcentered = x_new - np.average(x_new)
                ycentered = y_new - np.average(y_new)

                a = np.arctan2(-ycentered, xcentered)

                # bin collected info on radii of curvature, velocity, x and y positions as function of angle from center of TEM
                m, edge, binnum = binned_statistic(a[np.isfinite(r)], r[np.isfinite(r)], statistic = 'mean', bins = np.linspace(-math.pi, math.pi, angular_resolution + 1))
                curvs.append(m)

                m, edge, binnum = binned_statistic(a[np.isfinite(v)], v[np.isfinite(v)], statistic = 'mean', bins = np.linspace(-math.pi, math.pi, angular_resolution + 1))
                velos.append(m)

                m, edge, binnum = binned_statistic(a, x_new, statistic = 'mean', bins = np.linspace(-math.pi, math.pi, angular_resolution + 1))
                x_positions.append(m)

                m, edge, binnum = binned_statistic(a, y_new, statistic = 'mean', bins = np.linspace(-math.pi, math.pi, angular_resolution + 1))
                y_positions.append(m)

                if t < len(tem_exists_in_frame) - 1:
                    plt.scatter(x_new, y_new, c = r, vmin = -4, vmax = 4, cmap = 'coolwarm', alpha = 0.5)

        # WRC data must be collected a little differently than the geometric data above
        # by creating a donut-shaped region around the boundary of the TEM
        contracted = morphology.binary_erosion(tem_bin_map[t], selem = morphology.disk(2))
        expanded = morphology.binary_dilation(tem_bin_map[t], selem = morphology.disk(5))
        ring = expanded & ~contracted

        # and collecting angles between points in the ring and the center of the TEM
        y_vals_ring, x_vals_ring = np.where(ring)
        x_L = np.average(x_vals_ring.astype(float))
        y_L = np.average(y_vals_ring.astype(float))
        x_vals_centered = x_vals_ring.astype(float) - x_L
        y_vals_centered = y_vals_ring.astype(float) - y_L

        angle = np.arctan2(-y_vals_centered, x_vals_centered)

        # we can summarize the angularly resolved WRC signal around the edge of the TEM
        m, edge, binnum = binned_statistic(angle, wave_stack[t][ring] - wrc_bg, statistic = 'mean', bins = np.linspace(-math.pi, math.pi, angular_resolution + 1))
        s, edge, binnum = binned_statistic(angle, wave_stack[t][ring] - wrc_bg, statistic = 'std', bins = np.linspace(-math.pi, math.pi, angular_resolution + 1))
        fluor.append(m)

        m, edge, binnum = binned_statistic(angle, memb_stack[t][ring] - mem_bg, statistic = 'mean', bins = np.linspace(-math.pi, math.pi, angular_resolution + 1))
        s, edge, binnum = binned_statistic(angle, memb_stack[t][ring] - mem_bg, statistic = 'std', bins = np.linspace(-math.pi, math.pi, angular_resolution + 1))
        memcon.append(m)

    # convert collected data into numpy array for convenience
    curvs = np.array(curvs)
    velos = np.array(velos)
    fluor = np.array(fluor)
    memcon = np.array(memcon)

    plt.axis('off')

    plt.show()

    plt.subplot(111)
    plt.title('Spatiotemporal Dynamics\nof Membrane Signal on TEM of Interest', fontweight = 'bold')
    plt.imshow(memcon, vmin = 0, vmax = 100, origin = 'lower', aspect = 1, interpolation = 'nearest')
    plt.colorbar(shrink = 0.3, label = 'Membrane Siganl Intensity\n(above background)')

    xtick_pos = np.arange(0, angular_resolution + 1, angular_resolution/10) - 0.5
    xtick_lab = np.linspace(0, 360, len(xtick_pos))
    plt.xticks(xtick_pos, xtick_lab, rotation = 'vertical')
    plt.show()

    plt.figure(figsize = (10, 6), facecolor = 'w')

    plt.subplot(131)
    plt.title('Spatiotemporal Dynamics\nof WRC on TEM of Interest', fontweight = 'bold')
    plt.imshow(fluor, vmin = 0, vmax = 40, origin = 'lower', aspect = 1, interpolation = 'nearest')
    plt.colorbar(shrink = 0.3, label = 'WRC Intensity\n(above background)')

    xtick_pos = np.arange(0, angular_resolution + 1, angular_resolution/10) - 0.5
    xtick_lab = np.linspace(0, 360, len(xtick_pos))
    plt.xticks(xtick_pos, xtick_lab, rotation = 'vertical')

    plt.xlabel('Angle from Center\n(Degrees)', fontweight = 'bold')
    plt.ylabel('Time (frames)', fontweight = 'bold')

    plt.subplot(132)
    plt.title('Spatiotemporal Dynamics\nof Curvature on TEM of Interest', fontweight = 'bold')
    plt.imshow(curvs, vmin = -4, vmax = 4, cmap = 'coolwarm', origin = 'lower', aspect = 1, interpolation = 'nearest')
    plt.colorbar(shrink = 0.3, label = '1/Radius of Curvature (1/um)')

    xtick_pos = np.arange(0, angular_resolution + 1, angular_resolution/10) - 0.5
    xtick_lab = np.linspace(0, 360, len(xtick_pos))
    plt.xticks(xtick_pos, xtick_lab, rotation = 'vertical')
    plt.xlabel('Angle from Center\n(Degrees)', fontweight = 'bold')
    plt.ylabel('Time (frames)', fontweight = 'bold')

    plt.subplot(133)
    plt.title('Spatiotemporal Dynamics\nof Velocity on TEM of Interest', fontweight = 'bold')
    plt.imshow(velos, vmin = -10, vmax = 10, cmap = 'coolwarm', origin = 'lower', aspect = 1, interpolation = 'nearest')
    plt.colorbar(shrink = 0.3, label = 'Velocity (microns/minute)')

    xtick_pos = np.arange(0, angular_resolution + 1, angular_resolution/10) - 0.5
    xtick_lab = np.linspace(0, 360, len(xtick_pos))
    plt.xticks(xtick_pos, xtick_lab, rotation = 'vertical')
    plt.xlabel('Angle from Center\n(Degrees)', fontweight = 'bold')
    plt.ylabel('Time (frames)', fontweight = 'bold')

    plt.tight_layout()
    plt.show()

    dc = np.diff(curvs, axis = 0)
    m = memcon[:-1]
    w = fluor[:-1]
    c = curvs[:-1]
    

    b = np.linspace(0, 61, 31)

    plt.hist(w[(c > 0) & (dc < 0)], bins = b, density = True, color = 'blue', alpha = 0.5)
    plt.hist(w[(c > 0) & (dc > 0)], bins = b, density = True, color = 'red', alpha = 0.5)
    plt.show()

    fold_enrich = np.nanmean(w[(c > 0) & (dc < 0)]) / np.nanmean(w[(c > 0) & (dc > 0)])
    print(fold_enrich)
    
    a = np.log10(w[(c > 0) & (dc < 0)])
    b = np.log10(w[(c > 0) & (dc > 0)])
    print(stats.ttest_ind(a[np.isfinite(a)], b[np.isfinite(b)]))


    T, AR = w.shape


    fold_rot_func_3 = []
    fold_rot_func_3_control = []

    for i in range(-AR//2, AR//2 + 1):
        w_rotated = np.roll(w, i, axis = 1)
        m_rotated = np.roll(m, i, axis = 1)

        fold_enrich_null = np.nanmean(w_rotated[(c > 0) & (dc < 0)]) / np.nanmean(w_rotated[(c > 0) & (dc > 0)])
        fold_rot_func_3.append(fold_enrich_null)

        fold_enrich_null_control = np.nanmean(m_rotated[(c > 0) & (dc < 0)]) / np.nanmean(m_rotated[(c > 0) & (dc > 0)])
        fold_rot_func_3_control.append(fold_enrich_null_control)

    plt.axhline(1, ls = '--', color = 'k')
    plt.plot(np.linspace(-180, 180, AR + 1), fold_rot_func_3, label = 'WRC')
    plt.plot(np.linspace(-180, 180, AR + 1), fold_rot_func_3_control, label = 'Membrane')
    plt.ylabel('Fold Enrichment of Signal in\nflattening vs. lagging regions', fontweight = 'bold', fontsize = 15)
    plt.xlabel('Rotation', fontweight = 'bold', fontsize = 15)
    plt.xlim(-180, 180)
    plt.ylim(0.8, 1.2)
    plt.legend()
    plt.show()

    results.append([fold_rot_func_3, fold_rot_func_3_control])
    
    plt.subplot(111)
    plt.title('Spatiotemporal Dynamics\nof Membrane Signal on TEM of Interest', fontweight = 'bold')
    plt.imshow(fluor/memcon, vmin = 0, vmax = 1, origin = 'lower', aspect = 1, interpolation = 'nearest')
    plt.colorbar(shrink = 0.3, label = 'Membrane-Normalized WRC Signal Intensity\n(above background)')

    xtick_pos = np.arange(0, angular_resolution + 1, angular_resolution/10) - 0.5
    xtick_lab = np.linspace(0, 360, len(xtick_pos))
    plt.xticks(xtick_pos, xtick_lab, rotation = 'vertical')
    plt.show()

    plt.figure(figsize = (6,6))
    plt.title('Spatiotemporal Dynamics\nof Curvature on TEM of Interest', fontweight = 'bold')
    plt.imshow(curvs, vmin = -4, vmax = 4, cmap = 'coolwarm', origin = 'lower', aspect = 1)
    plt.colorbar(shrink = 0.5, label = 'Curvature')

    wave_bin_map = filters.gaussian(fluor, preserve_range = True) > 15
    plt.contour(wave_bin_map, levels = [False], colors = 'k', linestyles = '--')
    plt.contourf(wave_bin_map, levels = [0.5, 1], hatches = ['..'], colors='none')

    xtick_pos = np.arange(0, angular_resolution + 1, angular_resolution/10) - 0.5
    xtick_lab = np.linspace(0, 360, len(xtick_pos))
    plt.xticks(xtick_pos, xtick_lab, rotation = 'vertical')
    plt.xlabel('Angle from Center\n(Degrees)', fontweight = 'bold')
    plt.ylabel('Time (frames)', fontweight = 'bold')
    plt.tight_layout()
    plt.show()

In [ ]:
plt.scatter(fluor[1:-1], velos[1:-1], marker = '.', alpha = 0.1, color = 'C0')
plt.ylabel('Edge Velocity', fontweight = 'bold', fontsize = 15)
plt.xlabel('WRC Enrichment', fontweight = 'bold', fontsize = 15)
plt.axhline(0, ls = '--', color = 'k')
plt.ylim(-15,15)
plt.xlim(-10, 70)
plt.show()

plt.scatter(memcon[1:-1], velos[1:-1], marker = '.', alpha = 0.1, color = 'C0')
plt.ylabel('Edge Velocity', fontweight = 'bold', fontsize = 15)
plt.xlabel('Membrane Signal Enrichment', fontweight = 'bold', fontsize = 15)
plt.ylim(-15,15)
plt.xlim(-10, 350)
plt.show()

plt.scatter(fluor[1:-1]/memcon[1:-1], velos[1:-1], marker = '.', alpha = 0.1, color = 'C0')
plt.ylabel('Edge Velocity', fontweight = 'bold', fontsize = 15)
plt.xlabel('Normalized WRC Signal Enrichment', fontweight = 'bold', fontsize = 15)
plt.ylim(-15,15)
plt.xlim(-0.2, 1)
plt.show()

x = fluor[1:-1]/memcon[1:-1]
y = velos[1:-1]
print(spearmanr(x[np.isfinite(x) * np.isfinite(y)],y[np.isfinite(x) * np.isfinite(y)] ))

tem2_x = x
tem2_y = y

w = fluor[1:-1]/memcon[1:-1]
v = velos[1:-1]

T, AR = w.shape
fold_rot_func = []
for i in range(-AR//2, AR//2 + 1):
    w_rotated = np.roll(w, i, axis = 1)
    
    x = w_rotated
    y = v
    
    spearman_corr = spearmanr(x[np.isfinite(x) * np.isfinite(y)],y[np.isfinite(x) * np.isfinite(y)] )[0]
    fold_rot_func.append(spearman_corr)
    
    
plt.plot(np.linspace(-180, 180, AR + 1), fold_rot_func)
plt.ylabel('Spearman Correlation', fontweight = 'bold', fontsize = 15)
plt.xlabel('Rotation of Wave Complex Signal', fontweight = 'bold', fontsize = 15)
plt.xlim(-180, 180)
plt.ylim(-1, 1)
plt.show()

## TEM 3

In [ ]:
for t in range(0, 90, 5):
    plt.figure(figsize = (10,5))
    ax = plt.subplot(121)
    ymin = 100
    xmin = 520
    plt.imshow(im_1[t, 0], vmin = 100, vmax = 500)
    rect = mp.Rectangle((xmin, ymin), 80, 80, edgecolor = 'white', fill = None)
    ax.add_patch(rect)
    #plt.grid()
    
    plt.subplot(122)
    plt.imshow(im_1[t, 0, ymin:ymin + 80, xmin:xmin + 80], vmin = 100, vmax = 500)
    
    plt.tight_layout()
    plt.show()

In [ ]:
# zoom in on a closing TEM

count = 1

thresh = 140

plt.figure(figsize = (10,8))

for t in range(0, 100, 5): # look at every 5th frame
    ax = plt.subplot(4, 5, count)
    
    im_1[t, 0, ]
    crop = im_1[t, 0, ymin:ymin+80, xmin:xmin+80] # zoom in on one region
    crop_smth = filters.gaussian(crop, preserve_range = True)
    plt.imshow(crop, vmin = 100, vmax = 400)
    ctrs = measure.find_contours(crop_smth, thresh)

    for i in ctrs:
        plt.plot(i[:,1], i[:,0], color = 'w')
        
    ax.text(10, 10, 'F' + str(t), color = 'r', size = 12, fontweight = 'bold', ha = 'center', va = 'center')
    plt.xticks([])
    plt.yticks([])
    count += 1
    
plt.tight_layout()
plt.show()

# zoom in on a closing TEM

count = 1

plt.figure(figsize = (10,8))

for t in range(0, 100, 5): # look at every 5th frame
    ax = plt.subplot(4, 5, count)
    

    crop = im_1[t, 0, ymin:ymin+80, xmin:xmin+80] # zoom in on one region
    crop_smth = filters.gaussian(crop, preserve_range = True)
    crop2 = im_1[t, 1, ymin:ymin+80, xmin:xmin+80]
    plt.imshow(crop2, vmin = 100, vmax = 160)
    ctrs = measure.find_contours(crop_smth, thresh)

    for i in ctrs:
        plt.plot(i[:,1], i[:,0], color = 'w')
        
    ax.text(10, 10, 'F' + str(t), color = 'r', size = 12, fontweight = 'bold', ha = 'center', va = 'center')
    plt.xticks([])
    plt.yticks([])
    count += 1
    
plt.tight_layout()
plt.show()

smoothed_stack = [filters.gaussian(crop, preserve_range = True)[ymin:ymin+80, xmin:xmin+80] for crop in im_1[:,0]]
smoothed_stack = np.array(smoothed_stack)
crop_bin = (smoothed_stack < thresh)

crop_lbl = measure.label(crop_bin)

wave_stack = im_1[:, 1, ymin:ymin+80, xmin:xmin+80]


In [ ]:

for t in range(0, 100, 5):
    
    plt.figure(figsize = (5,5))
    ax = plt.subplot()

    reg_props = measure.regionprops(crop_lbl[t])

    plt.imshow(im_1[t, 0, ymin:ymin+80, xmin:xmin+80], vmin = 100, vmax = 400)

    ctrs = measure.find_contours(smoothed_stack[t], thresh)

    for i in ctrs:
        plt.plot(i[:,1], i[:,0], color = 'w')

    for c in range(len(reg_props)):
        L = reg_props[c].label
        y, x = reg_props[c].centroid

        ax.text(x, y, L, color = 'w', fontweight = 'bold')

    plt.axis('off')

    plt.show()

In [ ]:
TEM_OF_INTEREST = 2

tem_bin_map = np.array([crop_lbl[t] == TEM_OF_INTEREST for t in range(len(crop_lbl))])
tem_exists_in_frame = np.where([np.sum(crop_lbl[t] == TEM_OF_INTEREST) > 0 for t in range(len(crop_lbl))])[0]

# get coordinates for the center of the TEM over time
closing_coords = [center_of_mass(tem_bin_map[t]) for t in range(len(tem_bin_map))]

yc, xc = np.round(np.nanmean(closing_coords, axis = 0)).astype(int)

for t in tem_exists_in_frame:
    
    plt.figure(figsize = (10,5))
    ax = plt.subplot(121)

    reg_props = measure.regionprops(crop_lbl[t])

    plt.imshow(im_1[t, 0, ymin:ymin+80, xmin:xmin+80], vmin = 100, vmax = 400)

    ctrs = measure.find_contours(smoothed_stack[t], 130)

    for i in ctrs:
        x_m = np.average(i[:,1])
        y_m = np.average(i[:,0])


        y_L, x_L = closing_coords[t]

        if np.hypot(y_m - y_L, x_m - x_L) < 10:
            plt.plot(i[:,1], i[:,0], color = 'w', lw = 2, alpha = 0.8)
            
    rect = mp.Rectangle((1, 1), 5/microns_per_pixel, 2, color = 'white')
    #ax.text(15 , 7, '5 um', fontsize = 16, color = 'white', fontweight = 'bold', ha = 'center', va = 'center')
    #ax.add_patch(rect)
    plt.axis([xc - 50, xc + 50, yc - 50, yc + 50])
    plt.axis('off')
    
    ax2 = plt.subplot(122)
    
    plt.imshow(im_1[t, 1, ymin:ymin+80, xmin:xmin+80], vmin = 100, vmax = 160)

    ctrs = measure.find_contours(smoothed_stack[t], 130)

    for i in ctrs:
        x_m = np.average(i[:,1])
        y_m = np.average(i[:,0])


        y_L, x_L = closing_coords[t]

        if np.hypot(y_m - y_L, x_m - x_L) < 10:
            plt.plot(i[:,1], i[:,0], color = 'w', lw = 2, alpha = 0.8)
            
    plt.axis([xc - 50, xc + 50, yc - 50, yc + 50])
    #ax2.text(74, 74, str(np.round(timestamps_190326[t], 1)) + ' s', fontsize = 16, color = 'white', fontweight = 'bold', ha = 'right', va = 'center')
    plt.axis('off')

    plt.tight_layout()
    plt.show()

    #plt.savefig('/Users/jason/Desktop/Curvature_Anne/im2_tem_' + str(TEM_OF_INTEREST) + '/' + str(t).zfill(3) + '.png', bbox_inches = 'tight')
    #plt.close()

In [ ]:
TEM_OF_INTEREST = 2
closing_coords = [center_of_mass(crop_lbl[t] == TEM_OF_INTEREST) for t in range(99)]


for t in range(90):
    
    plt.figure(figsize = (9, 3))
    
    ax = plt.subplot(131)
    plt.imshow(im_1[t, 0], vmin = 100, vmax = 600)
    plt.xticks([])
    plt.yticks([])
    rect = mp.Rectangle((xmin, ymin), 80, 80, edgecolor = 'white', fill = None)
    ax.add_patch(rect)
    
    rect = mp.Rectangle((10, 10), 40/microns_per_pixel, 16, color = 'white')
    ax.text(130 , 75, '40 um', fontsize = 16, color = 'white', fontweight = 'bold', ha = 'center', va = 'center')
    ax.add_patch(rect)
    
    ax = plt.subplot(132)

    reg_props = measure.regionprops(crop_lbl[t])

    plt.imshow(im_1[t, 0, ymin:ymin+80, xmin:xmin+80], vmin = 100, vmax = 600)

    ctrs = measure.find_contours(smoothed_stack[t], thresh)

    for i in ctrs:
        x_m = np.average(i[:,1])
        y_m = np.average(i[:,0])


        y_L, x_L = closing_coords[t]
        y_Lb, x_Lb = closing_coords_b[t]

        if np.hypot(y_m - y_L, x_m - x_L) < 5:
            plt.plot(i[:,1], i[:,0], color = 'w', lw = 2, alpha = 0.8)

    rect = mp.Rectangle((1, 1), 5/microns_per_pixel, 2, color = 'white')
    ax.text(15 , 7, '5 um', fontsize = 16, color = 'white', fontweight = 'bold', ha = 'center', va = 'center')
    ax.add_patch(rect)
        
    plt.axis('off')
    
    ax2 = plt.subplot(133)
    
    plt.imshow(im_1[t, 1,  ymin:ymin+80, xmin:xmin+80], vmin = 100, vmax = 160)

    ctrs = measure.find_contours(smoothed_stack[t], thresh)

    for i in ctrs:
        x_m = np.average(i[:,1])
        y_m = np.average(i[:,0])


        y_L, x_L = closing_coords[t]
        y_Lb, x_Lb = closing_coords_b[t]

        if np.hypot(y_m - y_L, x_m - x_L) < 10:
            plt.plot(i[:,1], i[:,0], color = 'w', lw = 2, alpha = 0.8)
            
        #if np.hypot(y_m - y_Lb, x_m - x_Lb) < 10:
            #plt.plot(i[:,1], i[:,0], color = 'w', linestyle = '--', lw = 2, alpha = 0.8)
            
    
    ax2.text(74, 74, str(np.round(t * 5, 1)) + ' s', fontsize = 16, color = 'white', fontweight = 'bold', ha = 'right', va = 'center')
    plt.axis('off')

    plt.tight_layout()
    
    plt.show()
    


In [ ]:
TEM_OF_INTEREST = 2

# We can now easily extract a temporal profile of the area of our TEM of interest
area_tem = [np.sum(crop_lbl[t] == TEM_OF_INTEREST) for t in range(len(crop_lbl))]

# approximate the background fluorescence in the wrc channel
wrc_bg = np.median(wave_stack[crop_lbl == TEM_OF_INTEREST])

# We can also extract information about the fluorescence around our TEM of interest
fluor_tem = []

tem_present = np.where([np.sum(crop_lbl[t] == TEM_OF_INTEREST) > 0 for t in range(len(crop_lbl))])[0]
for t in tem_present:
    
    hole = (crop_lbl[t] == TEM_OF_INTEREST)
    
    if np.sum(hole) > 0:
        contracted = morphology.binary_erosion(hole, selem = morphology.disk(2))
        expanded = morphology.binary_dilation(hole, selem = morphology.disk(5))
        ring = expanded & ~contracted
        fluor_tem.append(np.average(wave_stack[t][ring]))
        
# take median intensity value inside the TEM as camera background value for WRC fluorescence
# a better background value might be taken from further inside the cell, but this would require more assumptions
fluor_bg = np.nanmedian([np.median(wave_stack[t][crop_lbl[t] == 1]) for t in range(99)])

plt.figure(figsize = (10, 4))
ax = plt.subplot()
timepoints_sec = time_per_frame_im1 * np.arange(len(area_tem))
plt.plot(timepoints_sec, np.array(area_tem) * microns_per_pixel**2, lw = 2, color = 'C1')
plt.ylabel('Area (Square Microns)', fontweight = 'bold', fontsize = 16, color = 'C1')
plt.xlabel('Time (Seconds)', fontweight = 'bold', fontsize = 16)
plt.ylim(0, 20)

ax.twinx()

plt.plot(timepoints_sec[tem_present], np.array(fluor_tem) - fluor_bg, lw = 2, color = 'C2')
plt.ylabel('Avg WRC Fluorescence\n(Units above background)', fontweight = 'bold', fontsize = 16, color = 'C2')
plt.ylim(0, 25)

plt.show()

In [ ]:


for TEM_OF_INTEREST in [2]:
    #########################
    # Parameters!
    #########################

    angular_resolution = 50
    #########################

    wave_stack = im_1[:, 1, ymin:ymin+80, xmin:xmin+80]
    memb_stack = im_1[:, 0, ymin:ymin+80, xmin:xmin+80]

    # approximate the background fluorescence in the wrc channel
    wrc_bg = np.median(wave_stack[crop_lbl == TEM_OF_INTEREST])
    mem_bg = np.median(memb_stack[crop_lbl == TEM_OF_INTEREST])

    # collect a binary map and time coordinates of when tem exists
    tem_bin_map = np.array([crop_lbl[t] == TEM_OF_INTEREST for t in range(len(crop_lbl))])
    tem_exists_in_frame = np.where([np.sum(crop_lbl[t] == TEM_OF_INTEREST) > 0 for t in range(len(crop_lbl))])[0]

    # get coordinates for the center of the TEM over time
    closing_coords = [center_of_mass(tem_bin_map[t]) for t in range(len(tem_bin_map))]

    # we also have the option to stabilize the TEM in case translational movement confounds the results
    #centered_bin_map = [np.roll(tem_bin_map[t], 40 - np.round(closing_coords[t][0]).astype(int), axis = 0) for t in tem_exists_in_frame]
    #centered_bin_map = [np.roll(centered_bin_map[t], 40 - np.round(closing_coords[t][1]).astype(int), axis = 1) for t in tem_exists_in_frame]
    #centered_bin_map = np.array(centered_bin_map)

    vmap = -generate_velmap(tem_bin_map.astype(int))
    t, s1, s2 = crop_lbl.shape

    angles = []
    curvs = []
    velos = []
    fluor = []
    x_positions = []
    y_positions = []
    memcon = []

    plt.figure(figsize = (10,10), facecolor = 'w')
    plt.subplot(aspect = 'equal')

    cmap = cm.get_cmap('viridis')
    col_vec = [cmap(i) for i in np.linspace(0,1,99)]

    for t in tem_exists_in_frame:

        # find smooth contours using the find_contours function
        ctrs = measure.find_contours(smoothed_stack[t], thresh)

        for i in ctrs:
            x_m = np.average(i[:,1])
            y_m = np.average(i[:,0])

            y_L, x_L = closing_coords[t]

            # find the smooth contour that corresponds to the TEM of interest (which is stored as a binary image)
            # note that the find_contours function has jagged features when applied to binary images. However,
            # these binary maps are *generally* easier to work with, hence the switching between the two
            if np.hypot(y_m - y_L, x_m - x_L) < 5:

                y, x = i.T

                # add extra points to our curve without changing its shape too much
                tck, u = interpolate.splprep([x, y], s = 0)
                param_new = np.linspace(0, 1, len(x) * 5) 
                x_new, y_new = interpolate.splev(param_new, tck)

                # find the radii of curvature along these newly created point-rich paths
                r, xcs, ycs = radius_of_curvature(x_new * microns_per_pixel, y_new * microns_per_pixel, 10)
                r = -np.array(r) # flip sign due to reversed orientation of segmented region compared to lamellipod

                # estimate velocity along the point-rich path from the minimum distances between edges at 
                # consecutive timepoints in microns/minute
                v = interpolate.interpn((np.arange(s1), np.arange(s2)), vmap[t] * microns_per_pixel/time_per_frame_im1 * 60, np.array([y_new, x_new]).T)

                # translate points to origin for angle calculations
                xcentered = x_new - np.average(x_new)
                ycentered = y_new - np.average(y_new)

                a = np.arctan2(-ycentered, xcentered)

                # bin collected info on radii of curvature, velocity, x and y positions as function of angle from center of TEM
                m, edge, binnum = binned_statistic(a[np.isfinite(r)], r[np.isfinite(r)], statistic = 'mean', bins = np.linspace(-math.pi, math.pi, angular_resolution + 1))
                curvs.append(m)

                m, edge, binnum = binned_statistic(a[np.isfinite(v)], v[np.isfinite(v)], statistic = 'mean', bins = np.linspace(-math.pi, math.pi, angular_resolution + 1))
                velos.append(m)

                m, edge, binnum = binned_statistic(a, x_new, statistic = 'mean', bins = np.linspace(-math.pi, math.pi, angular_resolution + 1))
                x_positions.append(m)

                m, edge, binnum = binned_statistic(a, y_new, statistic = 'mean', bins = np.linspace(-math.pi, math.pi, angular_resolution + 1))
                y_positions.append(m)

                if t < len(tem_exists_in_frame) - 1:
                    plt.scatter(x_new, y_new, c = r, vmin = -4, vmax = 4, cmap = 'coolwarm', alpha = 0.5)

        # WRC data must be collected a little differently than the geometric data above
        # by creating a donut-shaped region around the boundary of the TEM
        contracted = morphology.binary_erosion(tem_bin_map[t], selem = morphology.disk(2))
        expanded = morphology.binary_dilation(tem_bin_map[t], selem = morphology.disk(5))
        ring = expanded & ~contracted

        # and collecting angles between points in the ring and the center of the TEM
        y_vals_ring, x_vals_ring = np.where(ring)
        x_L = np.average(x_vals_ring.astype(float))
        y_L = np.average(y_vals_ring.astype(float))
        x_vals_centered = x_vals_ring.astype(float) - x_L
        y_vals_centered = y_vals_ring.astype(float) - y_L

        angle = np.arctan2(-y_vals_centered, x_vals_centered)

        # we can summarize the angularly resolved WRC signal around the edge of the TEM
        m, edge, binnum = binned_statistic(angle, wave_stack[t][ring] - wrc_bg, statistic = 'mean', bins = np.linspace(-math.pi, math.pi, angular_resolution + 1))
        s, edge, binnum = binned_statistic(angle, wave_stack[t][ring] - wrc_bg, statistic = 'std', bins = np.linspace(-math.pi, math.pi, angular_resolution + 1))
        fluor.append(m)

        m, edge, binnum = binned_statistic(angle, memb_stack[t][ring] - mem_bg, statistic = 'mean', bins = np.linspace(-math.pi, math.pi, angular_resolution + 1))
        s, edge, binnum = binned_statistic(angle, memb_stack[t][ring] - mem_bg, statistic = 'std', bins = np.linspace(-math.pi, math.pi, angular_resolution + 1))
        memcon.append(m)

    # convert collected data into numpy array for convenience
    curvs = np.array(curvs)
    velos = np.array(velos)
    fluor = np.array(fluor)
    memcon = np.array(memcon)

    plt.axis('off')

    plt.show()

    plt.subplot(111)
    plt.title('Spatiotemporal Dynamics\nof Membrane Signal on TEM of Interest', fontweight = 'bold')
    plt.imshow(memcon, vmin = 0, vmax = 100, origin = 'lower', aspect = 1, interpolation = 'nearest')
    plt.colorbar(shrink = 0.3, label = 'Membrane Siganl Intensity\n(above background)')

    xtick_pos = np.arange(0, angular_resolution + 1, angular_resolution/10) - 0.5
    xtick_lab = np.linspace(0, 360, len(xtick_pos))
    plt.xticks(xtick_pos, xtick_lab, rotation = 'vertical')
    plt.show()

    plt.figure(figsize = (10, 6), facecolor = 'w')

    plt.subplot(131)
    plt.title('Spatiotemporal Dynamics\nof WRC on TEM of Interest', fontweight = 'bold')
    plt.imshow(fluor, vmin = 0, vmax = 40, origin = 'lower', aspect = 1, interpolation = 'nearest')
    plt.colorbar(shrink = 0.3, label = 'WRC Intensity\n(above background)')

    xtick_pos = np.arange(0, angular_resolution + 1, angular_resolution/10) - 0.5
    xtick_lab = np.linspace(0, 360, len(xtick_pos))
    plt.xticks(xtick_pos, xtick_lab, rotation = 'vertical')

    plt.xlabel('Angle from Center\n(Degrees)', fontweight = 'bold')
    plt.ylabel('Time (frames)', fontweight = 'bold')

    plt.subplot(132)
    plt.title('Spatiotemporal Dynamics\nof Curvature on TEM of Interest', fontweight = 'bold')
    plt.imshow(curvs, vmin = -4, vmax = 4, cmap = 'coolwarm', origin = 'lower', aspect = 1, interpolation = 'nearest')
    plt.colorbar(shrink = 0.3, label = '1/Radius of Curvature (1/um)')

    xtick_pos = np.arange(0, angular_resolution + 1, angular_resolution/10) - 0.5
    xtick_lab = np.linspace(0, 360, len(xtick_pos))
    plt.xticks(xtick_pos, xtick_lab, rotation = 'vertical')
    plt.xlabel('Angle from Center\n(Degrees)', fontweight = 'bold')
    plt.ylabel('Time (frames)', fontweight = 'bold')

    plt.subplot(133)
    plt.title('Spatiotemporal Dynamics\nof Velocity on TEM of Interest', fontweight = 'bold')
    plt.imshow(velos, vmin = -10, vmax = 10, cmap = 'coolwarm', origin = 'lower', aspect = 1, interpolation = 'nearest')
    plt.colorbar(shrink = 0.3, label = 'Velocity (microns/minute)')

    xtick_pos = np.arange(0, angular_resolution + 1, angular_resolution/10) - 0.5
    xtick_lab = np.linspace(0, 360, len(xtick_pos))
    plt.xticks(xtick_pos, xtick_lab, rotation = 'vertical')
    plt.xlabel('Angle from Center\n(Degrees)', fontweight = 'bold')
    plt.ylabel('Time (frames)', fontweight = 'bold')

    plt.tight_layout()
    plt.show()

    dc = np.diff(curvs, axis = 0)
    m = memcon[:-1]
    w = fluor[:-1]
    c = curvs[:-1]
    

    b = np.linspace(0, 61, 31)

    plt.hist(w[(c > 0) & (dc < 0)], bins = b, density = True, color = 'blue', alpha = 0.5)
    plt.hist(w[(c > 0) & (dc > 0)], bins = b, density = True, color = 'red', alpha = 0.5)
    plt.show()

    
    fold_enrich = np.nanmean(w[(c > 0) & (dc < 0)]) / np.nanmean(w[(c > 0) & (dc > 0)])
    
    a = np.log10(w[(c > 0) & (dc < 0)])
    b = np.log10(w[(c > 0) & (dc > 0)])
    print(stats.ttest_ind(a[np.isfinite(a)], b[np.isfinite(b)]))
    
    print(fold_enrich)

    T, AR = w.shape


    fold_rot_func_3 = []
    fold_rot_func_3_control = []

    for i in range(-AR//2, AR//2 + 1):
        w_rotated = np.roll(w, i, axis = 1)
        m_rotated = np.roll(m, i, axis = 1)

        fold_enrich_null = np.nanmean(w_rotated[(c > 0) & (dc < 0)]) / np.nanmean(w_rotated[(c > 0) & (dc > 0)])
        fold_rot_func_3.append(fold_enrich_null)

        fold_enrich_null_control = np.nanmean(m_rotated[(c > 0) & (dc < 0)]) / np.nanmean(m_rotated[(c > 0) & (dc > 0)])
        fold_rot_func_3_control.append(fold_enrich_null_control)

    plt.axhline(1, ls = '--', color = 'k')
    plt.plot(np.linspace(-180, 180, AR + 1), fold_rot_func_3, label = 'WRC')
    plt.plot(np.linspace(-180, 180, AR + 1), fold_rot_func_3_control, label = 'Membrane')
    plt.ylabel('Fold Enrichment of Signal in\nflattening vs. lagging regions', fontweight = 'bold', fontsize = 15)
    plt.xlabel('Rotation', fontweight = 'bold', fontsize = 15)
    plt.xlim(-180, 180)
    plt.ylim(0.8, 1.2)
    plt.legend()
    plt.show()

    results.append([fold_rot_func_3, fold_rot_func_3_control])
    
    plt.subplot(111)
    plt.title('Spatiotemporal Dynamics\nof Membrane Signal on TEM of Interest', fontweight = 'bold')
    plt.imshow(fluor/memcon, vmin = 0, vmax = 1, origin = 'lower', aspect = 1, interpolation = 'nearest')
    plt.colorbar(shrink = 0.3, label = 'Membrane-Normalized WRC Signal Intensity\n(above background)')

    xtick_pos = np.arange(0, angular_resolution + 1, angular_resolution/10) - 0.5
    xtick_lab = np.linspace(0, 360, len(xtick_pos))
    plt.xticks(xtick_pos, xtick_lab, rotation = 'vertical')
    plt.show()

    plt.figure(figsize = (6,6))
    plt.title('Spatiotemporal Dynamics\nof Curvature on TEM of Interest', fontweight = 'bold')
    plt.imshow(curvs, vmin = -4, vmax = 4, cmap = 'coolwarm', origin = 'lower', aspect = 1)
    plt.colorbar(shrink = 0.5, label = 'Curvature')

    wave_bin_map = filters.gaussian(fluor, preserve_range = True) > 15
    plt.contour(wave_bin_map, levels = [False], colors = 'k', linestyles = '--')
    plt.contourf(wave_bin_map, levels = [0.5, 1], hatches = ['..'], colors='none')

    xtick_pos = np.arange(0, angular_resolution + 1, angular_resolution/10) - 0.5
    xtick_lab = np.linspace(0, 360, len(xtick_pos))
    plt.xticks(xtick_pos, xtick_lab, rotation = 'vertical')
    plt.xlabel('Angle from Center\n(Degrees)', fontweight = 'bold')
    plt.ylabel('Time (frames)', fontweight = 'bold')
    plt.tight_layout()
    plt.show()
    

In [ ]:

plt.scatter(fluor[1:-1], velos[1:-1], marker = '.', alpha = 0.1, color = 'C0')
plt.ylabel('Edge Velocity', fontweight = 'bold', fontsize = 15)
plt.xlabel('WRC Enrichment', fontweight = 'bold', fontsize = 15)
plt.axhline(0, ls = '--', color = 'k')
plt.ylim(-15,15)
plt.xlim(-10, 70)
plt.show()

plt.scatter(memcon[1:-1], velos[1:-1], marker = '.', alpha = 0.1, color = 'C0')
plt.ylabel('Edge Velocity', fontweight = 'bold', fontsize = 15)
plt.xlabel('Membrane Signal Enrichment', fontweight = 'bold', fontsize = 15)
plt.ylim(-15,15)
plt.xlim(-10, 350)
plt.show()

plt.scatter(fluor[1:-1]/memcon[1:-1], velos[1:-1], marker = '.', alpha = 0.1, color = 'C0')
plt.ylabel('Edge Velocity', fontweight = 'bold', fontsize = 15)
plt.xlabel('Normalized WRC Signal Enrichment', fontweight = 'bold', fontsize = 15)
plt.ylim(-15,15)
plt.xlim(-0.2, 1)
plt.show()

x = fluor[1:-1]/memcon[1:-1]
y = velos[1:-1]
print(spearmanr(x[np.isfinite(x) * np.isfinite(y)],y[np.isfinite(x) * np.isfinite(y)] ))


tem3_x = x
tem3_y = y

w = fluor[1:-1]/memcon[1:-1]
v = velos[1:-1]

T, AR = w.shape
fold_rot_func = []
for i in range(-AR//2, AR//2 + 1):
    w_rotated = np.roll(w, i, axis = 1)
    
    x = w_rotated
    y = v
    
    spearman_corr = spearmanr(x[np.isfinite(x) * np.isfinite(y)],y[np.isfinite(x) * np.isfinite(y)] )[0]
    fold_rot_func.append(spearman_corr)
    
    
plt.plot(np.linspace(-180, 180, AR + 1), fold_rot_func)
plt.ylabel('Spearman Correlation', fontweight = 'bold', fontsize = 15)
plt.xlabel('Rotation of Wave Complex Signal', fontweight = 'bold', fontsize = 15)
plt.xlim(-180, 180)
plt.ylim(-1, 1)
plt.show()

## TEMs Summary

In [ ]:
plt.figure(figsize = (12, 4))
plt.subplot(131)
plt.scatter(tem1_x, tem1_y, marker = '.', alpha = 0.2)
plt.ylabel('Edge Velocity', fontweight = 'bold', fontsize = 15)
plt.xlabel('Normalized WRC\nSignal Enrichment', fontweight = 'bold', fontsize = 15)
plt.title("TEM 1", fontweight = 'bold', fontsize = 15)

plt.subplot(132)
plt.scatter(tem2_x, tem2_y, marker = '.', alpha = 0.2)
plt.ylabel('Edge Velocity', fontweight = 'bold', fontsize = 15)
plt.xlabel('Normalized WRC\nSignal Enrichment', fontweight = 'bold', fontsize = 15)
plt.title("TEM 2", fontweight = 'bold', fontsize = 15)

plt.subplot(133)
plt.scatter(tem3_x, tem3_y, marker = '.', alpha = 0.2)
plt.ylabel('Edge Velocity', fontweight = 'bold', fontsize = 15)
plt.xlabel('Normalized WRC\nSignal Enrichment', fontweight = 'bold', fontsize = 15)
plt.title("TEM 3", fontweight = 'bold', fontsize = 15)

plt.tight_layout()
plt.show()



## HL60 Analysis
Further analysis of leading edge dynamics

In [ ]:
parent = '/Users/jason/Projects/wavecomplex_selforganization/data/raw_data/'
filename = '171106_HWC_fmlp_8_SIR_ALX-2.tif'
im = io.imread(parent + filename)
t, c, w, h = im.shape

In [ ]:
print(im.shape)

In [ ]:
microns_per_pixel = 29.04/654 
print(microns_per_pixel)

time_per_frame = 2.0


In [ ]:
t = 0
blur_im = filters.gaussian(im[t, 0], 2, preserve_range = True)
cell_mask = blur_im > 0.75 * filters.threshold_otsu(blur_im)
cell_mask = morphology.binary_erosion(cell_mask, selem = morphology.disk(5))
cell_mask = morphology.binary_dilation(cell_mask, selem = morphology.disk(2))
cell_mask = morphology.remove_small_holes(cell_mask, 2000)
cell_mask = morphology.remove_small_objects(cell_mask, 2000)

plt.imshow(im[0,0])
plt.colorbar()

In [ ]:
smoothed_stack = []
cell_bin_map = []
for t in range(50):
    blur_im = filters.gaussian(im[t, 0], 2, preserve_range = True)
    smoothed_stack.append(blur_im)
    cell_mask = blur_im > 0.75 * filters.threshold_otsu(blur_im)
    cell_mask = morphology.binary_erosion(cell_mask, selem = morphology.disk(5))
    cell_mask = morphology.binary_dilation(cell_mask, selem = morphology.disk(2))
    cell_mask = morphology.remove_small_holes(cell_mask, 2000)
    cell_mask = morphology.remove_small_objects(cell_mask, 2000)
    cell_bin_map.append(cell_mask)
    
    plt.imshow(im[t, 0])
    plt.contour(cell_mask, levels = [False], colors = 'w')
    plt.show()
    
smoothed_stack = np.array(smoothed_stack)
cell_bin_map = np.array(cell_bin_map)
wave_stack = im[:50,1]
memb_stack = im[:50, 0]

In [ ]:
#########################
# Parameters!
#########################
angular_resolution = 1000
#########################

# approximate the background fluorescence in the wrc channel
wrc_bg = np.median(im[:50,1][cell_bin_map == False])
memb_bg = np.median(im[:50,0][cell_bin_map == False])

# collect a binary map and time coordinates of when tem exists
tem_bin_map = cell_bin_map
tem_exists_in_frame = np.where([np.sum(cell_bin_map[t]) > 0 for t in range(len(cell_bin_map))])[0]

# get coordinates for the center of the TEM over time
closing_coords = [center_of_mass(tem_bin_map[t]) for t in tem_exists_in_frame]


vmap = generate_velmap(tem_bin_map.astype(int)) # sign here is positive due to topology
t, s1, s2 = tem_bin_map.shape

angles = []
curvs = []
velos = []
fluor = []
x_positions = []
y_positions = []
mem_con = []

plt.figure(figsize = (10,10), facecolor = 'w')
plt.subplot(aspect = 'equal')

cmap = cm.get_cmap('viridis')
col_vec = [cmap(i) for i in np.linspace(0,1,99)]

for t in tem_exists_in_frame:
    
    # find smooth contours using the find_contours function
    smth_bin = filters.gaussian(cell_bin_map[t].astype(float))
    ctrs = measure.find_contours(smth_bin, 0.5)

    for i in ctrs:
        x_m = np.average(i[:,1])
        y_m = np.average(i[:,0])

        y_L, x_L = closing_coords[t]
        
        # find the smooth contour that corresponds to the TEM of interest (which is stored as a binary image)
        # note that the find_contours function has jagged features when applied to binary images. However,
        # these binary maps are *generally* easier to work with, hence the switching between the two
        if np.hypot(y_m - y_L, x_m - x_L) < 500:

            y, x = i.T
            
            # add extra points to our curve without changing its shape too much
            tck, u = interpolate.splprep([x, y], s = 0)
            param_new = np.linspace(0, 1, len(x) * 5) 
            x_new, y_new = interpolate.splev(param_new, tck)
            
            # find the radii of curvature along these newly created point-rich paths
            r, xcs, ycs = radius_of_curvature(x_new * microns_per_pixel, y_new * microns_per_pixel, 44) # pixels are 4.4 fold different from tems data
            r = -np.array(r) # flip sign due to reversed orientation of segmented region compared to lamellipod
            
            # estimate velocity along the point-rich path from the minimum distances between edges at 
            # consecutive timepoints in microns/minute
            v = interpolate.interpn((np.arange(s1), np.arange(s2)), vmap[t] * microns_per_pixel/time_per_frame * 60, np.array([y_new, x_new]).T)
            
            
            # translate points to origin for angle calculations
            xcentered = x_new - np.average(x_new)
            ycentered = y_new - np.average(y_new)
            
            a = np.arctan2(-ycentered, xcentered)
            
            # bin collected info on radii of curvature, velocity, x and y positions as function of angle from center of TEM
            m, edge, binnum = binned_statistic(a[np.isfinite(r)], r[np.isfinite(r)], statistic = 'mean', bins = np.linspace(-math.pi, math.pi, angular_resolution + 1))
            curvs.append(m)
            
            m, edge, binnum = binned_statistic(a[np.isfinite(v)], v[np.isfinite(v)], statistic = 'mean', bins = np.linspace(-math.pi, math.pi, angular_resolution + 1))
            velos.append(m)
            
            m, edge, binnum = binned_statistic(a, x_new, statistic = 'mean', bins = np.linspace(-math.pi, math.pi, angular_resolution + 1))
            x_positions.append(m)
            
            m, edge, binnum = binned_statistic(a, y_new, statistic = 'mean', bins = np.linspace(-math.pi, math.pi, angular_resolution + 1))
            y_positions.append(m)
            
            if t < len(tem_exists_in_frame) - 1:
                plt.scatter(x_new, y_new, c = r, vmin = -4, vmax = 4, cmap = 'coolwarm', alpha = 0.5, marker = '.')
                
    # WRC data must be collected a little differently than the geometric data above
    # by creating a donut-shaped region around the boundary of the TEM
    contracted = morphology.binary_erosion(tem_bin_map[t], selem = morphology.disk(11)) # this corresponds to about 0.5 um
    expanded = morphology.binary_dilation(tem_bin_map[t], selem = morphology.disk(4)) # this corresponds to about 0.2 um
    ring = expanded & ~contracted
    
    # and collecting angles between points in the ring and the center of the TEM
    y_vals_ring, x_vals_ring = np.where(ring)
    x_L = np.average(x_vals_ring.astype(float))
    y_L = np.average(y_vals_ring.astype(float))
    x_vals_centered = x_vals_ring.astype(float) - x_L
    y_vals_centered = y_vals_ring.astype(float) - y_L

    angle = np.arctan2(-y_vals_centered, x_vals_centered)

    # we can summarize the angularly resolved WRC signal around the edge of the TEM
    m, edge, binnum = binned_statistic(angle, wave_stack[t][ring] - wrc_bg, statistic = 'mean', bins = np.linspace(-math.pi, math.pi, angular_resolution + 1))
    s, edge, binnum = binned_statistic(angle, wave_stack[t][ring] - wrc_bg, statistic = 'std', bins = np.linspace(-math.pi, math.pi, angular_resolution + 1))
    fluor.append(m)
    
    m, edge, binnum = binned_statistic(angle, memb_stack[t][ring] - memb_bg, statistic = 'mean', bins = np.linspace(-math.pi, math.pi, angular_resolution + 1))
    s, edge, binnum = binned_statistic(angle, memb_stack[t][ring] - memb_bg, statistic = 'std', bins = np.linspace(-math.pi, math.pi, angular_resolution + 1))
    mem_con.append(m)
    
# convert collected data into numpy array for convenience
curvs = np.array(curvs)
velos = np.array(velos)
fluor = np.array(fluor)
mem_con = np.array(mem_con)

            
plt.axis('off')

plt.show()



In [ ]:
A = np.linspace(0, 360, len(curvs[0]) + 1)
T = np.linspace(0, len(curvs) * 2, len(curvs))

plt.figure(figsize = (10,3))
plt.pcolormesh(A, T, curvs, vmin = -4, vmax = 4, cmap = 'coolwarm')
plt.xlabel('Angle from Centroid', fontweight = 'bold', fontsize = 15)
plt.ylabel('Time (seconds)', fontweight = 'bold', fontsize = 15)
plt.colorbar(shrink = 0.8, label = '1/radius of curvature (1/um)')
plt.tight_layout()
plt.show()

plt.figure(figsize = (10,3))
plt.pcolormesh(A, T, velos, vmin = -10, vmax = 10, cmap = 'coolwarm')
plt.xlabel('Angle from Centroid', fontweight = 'bold', fontsize = 15)
plt.ylabel('Time (seconds)', fontweight = 'bold', fontsize = 15)
plt.colorbar(shrink = 0.8, label = 'Edge velocity (um/min)')
plt.tight_layout()
plt.show()

plt.figure(figsize = (10,3))
plt.pcolormesh(A, T, fluor, vmin = 0, vmax = 100, cmap = 'viridis')
plt.xlabel('Angle from Centroid', fontweight = 'bold', fontsize = 15)
plt.ylabel('Time (seconds)', fontweight = 'bold', fontsize = 15)
plt.colorbar(shrink = 0.8, label = 'WRC Intensity')
plt.tight_layout()
plt.show()

plt.figure(figsize = (10,3))
plt.pcolormesh(A, T, mem_con, vmin = 0, vmax = 200, cmap = 'viridis')
plt.xlabel('Angle from Centroid', fontweight = 'bold', fontsize = 15)
plt.ylabel('Time (seconds)', fontweight = 'bold', fontsize = 15)
plt.colorbar(shrink = 0.8, label = 'Membrane Intensity')
plt.tight_layout()
plt.show()

In [ ]:

plt.scatter(fluor[1:-1], velos[1:-1], marker = '.', alpha = 0.1, color = 'C0')
plt.ylabel('Edge Velocity', fontweight = 'bold', fontsize = 15)
plt.xlabel('WRC Enrichment', fontweight = 'bold', fontsize = 15)
plt.axhline(0, ls = '--', color = 'k')
plt.ylim(-15,15)
plt.xlim(-10, 70)
plt.show()

plt.scatter(mem_con[1:-1], velos[1:-1], marker = '.', alpha = 0.1, color = 'C0')
plt.ylabel('Edge Velocity', fontweight = 'bold', fontsize = 15)
plt.xlabel('Membrane Signal Enrichment', fontweight = 'bold', fontsize = 15)
plt.ylim(-15,15)
plt.xlim(-10, 350)
plt.show()

plt.scatter(fluor[1:-1]/mem_con[1:-1], velos[1:-1], marker = '.', alpha = 0.1, color = 'C0')
plt.ylabel('Edge Velocity', fontweight = 'bold', fontsize = 15)
plt.xlabel('Normalized WRC Signal Enrichment', fontweight = 'bold', fontsize = 15)
plt.ylim(-15,15)
plt.xlim(-0.2, 1)
plt.show()

x = fluor[1:-1]/mem_con[1:-1]
y = velos[1:-1]
print(spearmanr(x[np.isfinite(x) * np.isfinite(y)],y[np.isfinite(x) * np.isfinite(y)] ))


w = fluor[1:-1]/mem_con[1:-1]
v = velos[1:-1]

T, AR = w.shape
fold_rot_func = []
for i in range(-AR//2, AR//2 + 1):
    w_rotated = np.roll(w, i, axis = 1)
    
    x = w_rotated
    y = v
    
    spearman_corr = spearmanr(x[np.isfinite(x) * np.isfinite(y)],y[np.isfinite(x) * np.isfinite(y)] )[0]
    fold_rot_func.append(spearman_corr)
    
    
plt.plot(np.linspace(-180, 180, AR + 1), fold_rot_func)
plt.ylabel('Spearman Correlation', fontweight = 'bold', fontsize = 15)
plt.xlabel('Rotation of Wave Complex Signal', fontweight = 'bold', fontsize = 15)
plt.xlim(-180, 180)
plt.ylim(-1, 1)
plt.show()

w = fluor[:-1]
c = curvs[:-1]
dc = np.diff(curvs, axis = 0)


b = np.linspace(0, 160, 31)

plt.hist(w[(c > 0) & (dc < 0)], bins = b, density = True, color = 'blue', alpha = 0.5)
plt.hist(w[(c > 0) & (dc > 0)], bins = b, density = True, color = 'red', alpha = 0.5)
plt.show()

fold_enrich = np.nanmean(w[(c > 0) & (dc < 0)]) / np.nanmean(w[(c > 0) & (dc > 0)])
print(fold_enrich)

T, AR = w.shape


fold_rot_func_4 = []
for i in range(-AR//2, AR//2 + 1):
    w_rotated = np.roll(w, i, axis = 1)
    fold_enrich_null = np.nanmean(w_rotated[(c > 0) & (dc < 0)]) / np.nanmean(w_rotated[(c > 0) & (dc > 0)])
    fold_rot_func_4.append(fold_enrich_null)
    
plt.axhline(1, ls = '--', color = 'k')
plt.plot(np.linspace(-180, 180, AR + 1), fold_rot_func_4)
plt.ylabel('Fold Enrichment of WRC in\nflattening vs. lagging regions', fontweight = 'bold', fontsize = 15)
plt.xlabel('Rotation', fontweight = 'bold', fontsize = 15)
plt.xlim(-180, 180)
plt.ylim(0.8, 1.2)
plt.show()

w = mem_con[:-1]
c = curvs[:-1]
dc = np.diff(curvs, axis = 0)

b = np.linspace(0, 160, 31)

plt.hist(w[(c > 0) & (dc < 0)], bins = b, density = True, color = 'blue', alpha = 0.5)
plt.hist(w[(c > 0) & (dc > 0)], bins = b, density = True, color = 'red', alpha = 0.5)
plt.show()

fold_enrich = np.nanmean(w[(c > 0) & (dc < 0)]) / np.nanmean(w[(c > 0) & (dc > 0)])
print(fold_enrich)

T, AR = w.shape


fold_rot_func_4_con = []
for i in range(-AR//2, AR//2 + 1):
    w_rotated = np.roll(w, i, axis = 1)
    fold_enrich_null = np.nanmean(w_rotated[(c > 0) & (dc < 0)]) / np.nanmean(w_rotated[(c > 0) & (dc > 0)])
    fold_rot_func_4_con.append(fold_enrich_null)
    
plt.axhline(1, ls = '--', color = 'k')
plt.plot(np.linspace(-180, 180, AR + 1), fold_rot_func_4_con)
plt.ylabel('Fold Enrichment of WRC in\nflattening vs. lagging regions', fontweight = 'bold', fontsize = 15)
plt.xlabel('Rotation', fontweight = 'bold', fontsize = 15)
plt.xlim(-180, 180)
plt.ylim(0.8, 1.2)
plt.show()

plt.axhline(1, ls = '--', color = 'k')
plt.plot(np.linspace(-180, 180, AR + 1), fold_rot_func_4, label = 'WRC')
plt.plot(np.linspace(-180, 180, AR + 1), fold_rot_func_4_con, label = 'Membrane')
plt.ylabel('Fold Enrichment of Signal in\nflattening vs. lagging regions', fontweight = 'bold', fontsize = 15)
plt.xlabel('Rotation', fontweight = 'bold', fontsize = 15)
plt.xlim(-180, 180)
plt.ylim(0.8, 1.2)
plt.legend()
plt.show()

In [ ]:
P = []

for R in range(51):
    stat, pval = stats.ttest_1samp(np.array(results)[:,0,R], 1)
    P.append(pval)

AR = len(results[0][0])
plt.axhline(1, ls = '--', color = 'k')

A = np.linspace(-180, 180, AR)

M0 = np.average([i[0] for i in results], axis = 0)
S0 = np.std([i[0] for i in results], axis = 0)
M1 = np.average([i[1] for i in results], axis = 0)
S1 = np.std([i[1] for i in results], axis = 0)

plt.plot(A, M0, zorder = 0, label = 'Wave Signal')
plt.fill_between(A, M0 - S0, M0 + S0, alpha = 0.2)
plt.plot(A, M1, zorder = 0, label = 'Membrane Signal')
plt.fill_between(A, M1 - S1, M1 + S1, alpha = 0.2)

plt.ylabel('Fold Enrichment of WRC in\nflattening vs. lagging regions', fontweight = 'bold', fontsize = 15)
plt.xlabel('Rotation', fontweight = 'bold', fontsize = 15)
plt.xlim(-180, 180)
plt.ylim(0.8, 1.2)

sig = np.array(P) < 0.05
plt.scatter(A[sig], M0[sig], marker = '*', color = 'k', zorder = 1, s = 100)

plt.legend()

plt.show()